## Modul 5 | TensorFlow

### Latihan Pembuatan Model Klasifikasi Gambar

In [1]:
# lib imports
import tensorflow as tf
print(tf.__version__)


2.15.0


In [5]:
!wget --no-check-certificate \
    https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip \
    -O /tmp/messy_vs_clean_room.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# melakukan ekstraksi pada file zip
import zipfile, os
local_zip = '/tmp/messy_vs_clean_room.zip'

In [7]:
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/messy_vs_clean_room.zip'